## Домашняя работа. Связь бизнес-показателей и DS-метрик

### Задания

1. Обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта

2. При обучении моделей обязательно использовать кроссвалидацию

3. Вывести сравнение полученных моделей по основным метрикам классификации: ``pr/rec/auc/f_score`` (можно в виде таблицы, где строки - модели, а столбцы - метрики)

4. Сделать выводы о том, какая модель справилась с задачей лучше других

5. (Опциональный вопрос) какой график (``precision_recall_curve`` или ``roc_auc_curve``) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого, например, 1 к 1000).

**p.s.** В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно).

Допустим, у нас две модели:

- Первая помечает 100 объектов как класс 1, но TP = 90
- Вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - ``roc_auc_curve`` или ``precision_recall_curve``?

**Ссылки:** 
- https://dyakonov.org/2017/07/28/auc-roc-площадь-под-кривой-ошибок/
- https://en.wikipedia.org/wiki/Receiver_operating_characteristic

---

In [1]:
# Загрузка необходимых библиотек

import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import FeatureUnion

from scipy.sparse import hstack

In [2]:
# Загрузка данных

df_train = pd.read_csv('train_case2.csv', ';')

# Проверка

df_train.head(10)

/Users/ilya.ivolgin/miniconda3/envs/geekbrains/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
5,8,21914,1,151,67.0,120,80,2,2,0,0,0,0
6,9,22113,1,157,93.0,130,80,3,1,0,0,1,0
7,12,22584,2,178,95.0,130,90,3,3,0,0,1,1
8,13,17668,1,158,71.0,110,70,1,1,0,0,1,0
9,14,19834,1,164,68.0,110,60,1,1,0,0,0,0


---

In [3]:
# Разделение на train и test

X_train, X_test, y_train, y_test = train_test_split(df_train.drop('cardio', axis = 1), df_train['cardio'], random_state = 42)

**Изменения:**
- `gender`, `cholesterol` - OHE
- `age`, `height`, `weight`, `ap_hi`, `ap_lo` - Standard
- `gluc`, `smoke`, `alco`, `active`- без изменений 

In [4]:
# Изменения

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        
        self.key = key

    def fit(self, X, y = None):
        
        return self

    def transform(self, X):
        
        return X[self.key]


class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        
        self.key = key

    def fit(self, X, y = None):
        
        return self

    def transform(self, X):
        
        return X[[self.key]]


class OHEEncoder(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        
        self.key = key
        self.columns = []

    def fit(self, X, y = None):
        
        self.columns = [
            col for col in pd.get_dummies(X, prefix=self.key).columns
        ]
        
        return self

    def transform(self, X):
        
        X = pd.get_dummies(X, prefix = self.key)
        test_columns = [col for col in X.columns]
        
        for col_ in test_columns:
            
            if col_ not in self.columns:
                
                X[col_] = 0
                
        return X[self.columns]


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

# Pipeline

for cont_col in continuos_cols:
    
    transfomer = Pipeline([('selector', NumberSelector(key=cont_col)),
                           ('standard', StandardScaler())])
    continuos_transformers.append((cont_col, transfomer))

for cat_col in cat_cols:
    
    cat_transformer = Pipeline([('selector', ColumnSelector(key=cat_col)),
                                ('ohe', OHEEncoder(key=cat_col))])
    cat_transformers.append((cat_col, cat_transformer))

for base_col in base_cols:
    
    base_transformer = Pipeline([('selector', NumberSelector(key=base_col))])
    base_transformers.append((base_col, base_transformer))

In [5]:
# FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)

feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[ 0.09144178,  0.44594423,  0.05752649, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.46216539,  0.20194394, -0.84752919, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.5702846 , -1.01805752,  1.79801819, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.67238553,  0.56794437, -0.01209318, ...,  0.        ,
         0.        ,  1.        ],
       [-0.50009534,  0.32394408, -0.29057185, ...,  0.        ,
         0.        ,  0.        ],
       [-1.76136594,  1.54394554, -0.70828985, ...,  0.        ,
         0.        ,  1.        ]])

In [6]:
# Cross validation

log_reg = Pipeline([('features', feats),
                    ('classifier', LogisticRegression(random_state = 42))])

In [7]:
# Scores for classifier

def scores_for_classifier(classifier):
    
    # Cross Val
    
    cv_scores = cross_val_score(classifier, X_train, y_train, cv = 16, scoring = 'roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)
    
    # Use train data for learning
    
    classifier.fit(X_train, y_train)
    y_score = classifier.predict_proba(X_test)[:, 1]
    
    # Metrics
    
    b = 1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1 + b**2) * (precision * recall) / (b**2 * precision + recall)
    roc = roc_auc_score(y_test, y_score)
    ix = np.argmax(fscore)
    
    return [round(i, 3) for i in [cv_score, thresholds[ix], fscore[ix], precision[ix], recall[ix], roc]]

In [8]:
%%time

results = pd.DataFrame([
    scores_for_classifier(classifier = Pipeline([('features', feats), ('classifier', LogisticRegression(random_state = 42))])),
    scores_for_classifier(classifier = Pipeline([('features', feats), ('classifier', RandomForestClassifier(random_state = 42))])),
    scores_for_classifier(classifier = Pipeline([('features', feats), ('classifier', DecisionTreeClassifier(max_depth = 4, random_state = 42))])),
], columns = ['CV_score', 'Best Threshold', 'F-Score', 'Precision', 'Recall', 'ROC AUC score'])

results['models'] = ['LogisticRegression', 'RandomForestClassifier', 'DecisionTreeClassifier']
results = results.set_index('models')

CPU times: user 1min 38s, sys: 1.91 s, total: 1min 39s
Wall time: 1min 32s


In [9]:
results

,CV_score,Best Threshold,F-Score,Precision,Recall,ROC AUC score
models,,,,,,
LogisticRegression,0.782,0.397,0.735,0.656,0.837,0.785
RandomForestClassifier,0.775,0.370,0.722,0.653,0.809,0.772
DecisionTreeClassifier,0.788,0.382,0.734,0.649,0.844,0.788


---

**Результат:**
- если анализировать только `ROC_AUC`, то лучшей моделью является `DecisionTree`
- отличие в результатах по `ROC_AUC` между `DecisionTree` и `LogisticRegression` является небольшим, возможны другие результаты при измененных настройках
- есть смысл сконцентрироваться на `Recall`